### All days of the challange:

* [Day 1: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Day 2: Scaling and normalization](https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data)
* [Day 3: Parsing dates](https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates/)
* [Day 4: Character encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/)
* [Day 5: Inconsistent Data Entry](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/)
___

¡Bienvenido al día 5 del Desafío de datos de 5 días! (¿Puedes creer que ya han pasado cinco días?) Hoy, vamos a aprender cómo limpiar entradas de texto inconsistentes. Para comenzar, haga clic en el botón azul "Fork Notebook" en la esquina superior derecha. Esto creará una copia privada de este cuaderno que puede editar y jugar. Una vez que haya terminado con los ejercicios, puede optar por hacer público su cuaderno para compartirlo con otros. :)

> **¡Tu turno!** Mientras trabajamos en este cuaderno, verá algunas celdas del cuaderno (un bloque de código o texto) que tiene "¡Tu turno!" escrito en él. Estos son ejercicios que puede realizar para ayudar a consolidar su comprensión de los conceptos de los que estamos hablando. Una vez que haya escrito el código para responder una pregunta específica, puede ejecutar el código haciendo clic dentro de la celda (cuadro con el código) con el código que desea ejecutar y luego presione CTRL + ENTER (CMD + ENTER en una Mac ). También puede hacer clic en una celda y luego hacer clic en la flecha derecha "reproducir" a la izquierda del código. Si desea ejecutar todo el código en su cuaderno, puede usar las flechas dobles de "avance rápido" en la parte inferior del editor del cuaderno.

Esto es lo que haremos hoy:

* [Get our environment set up](#Get-our-environment-set-up)
* [Do some preliminary text pre-processing](#Do-some-preliminary-text-pre-processing)
* [Use fuzzy matching to correct inconsistent data entry](#Use-fuzzy-matching-to-correct-inconsistent-data-entry)


¡Empecemos!

# Configurar nuestro entorno
________

Lo primero que tendremos que hacer es cargar las bibliotecas que usaremos. Sin embargo, no nuestros conjuntos de datos: ¡llegaremos a ellos más tarde!

> **¡Importante!** ¡Asegúrese de ejecutar esta celda usted mismo o el resto de su código no funcionará!

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
np.random.seed(0)

Cuando intenté leer en el archivo `PakistanSuicideAttacks Ver 11 (30-November-2017) .csv` por primera vez, recibí un error de codificación de caracteres, así que voy a comprobar rápidamente cuál debería ser la codificación ...

In [ ]:
# look at the first ten thousand bytes to guess the character encoding
with open("../input/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

Y luego léelo con la codificación correcta. (Si esto no le resulta familiar, consulte [yesterday's challenge](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/).) 

In [ ]:
# read in our dat
suicide_attacks = pd.read_csv("../input/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 
                              encoding='Windows-1252')

¡Ahora estamos listos para empezar! Como siempre, puede tomarse un momento para ver los datos y familiarizarse con ellos. :)


# Haga un preprocesamiento preliminar del texto
___

Para este ejercicio, estoy interesado en limpiar la columna "Ciudad" para asegurarme de que no haya inconsistencias en la entrada de datos. Podríamos revisar y verificar cada fila a mano, por supuesto, y corregir manualmente las inconsistencias cuando las encontremos. ¡Sin embargo, hay una forma más eficiente de hacer esto!

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

Con solo mirar esto, puedo ver algunos problemas debido a la entrada de datos inconsistente: 'Lahore' y 'Lahore', por ejemplo, o 'Lakki Marwat' y 'Lakki marwat'.

Lo primero que voy a hacer es poner todo en minúsculas (puedo volver a cambiarlo al final si quiero) y eliminar los espacios en blanco al principio y al final de las celdas. Las inconsistencias en las mayúsculas y los espacios en blanco finales son muy comunes en los datos de texto y puede corregir un buen 80% de las inconsistencias de entrada de datos de texto haciendo esto.

In [ ]:
# convert to lower case
suicide_attacks['City'] = suicide_attacks['City'].str.lower()
# remove trailing white spaces
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

A continuación, abordaremos inconsistencias más difíciles.

In [ ]:
# Your turn! Take a look at all the unique values in the "Province" column. 
# Then convert the column to lowercase and remove any trailing white spaces

# Utilice la coincidencia aproximada para corregir la entrada de datos inconsistentes
___

Muy bien, echemos otro vistazo a la columna de la ciudad y veamos si hay más limpieza de datos que debamos hacer.

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

Parece que quedan algunas inconsistencias: 'd. i khan 'y' d.i khan 'probablemente deberían ser lo mismo. (I [looked it up](https://en.wikipedia.org/wiki/List_of_most_populous_cities_in_Pakistan) y 'd.g khan' es una ciudad separada, así que no debería combinarlos).

Voy a usar el [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) paquete para ayudar a identificar qué cadenas están más cerca una de la otra. Este conjunto de datos es lo suficientemente pequeño como para que podamos corregir los errores a mano, pero ese enfoque no se escala bien. (¿Le gustaría corregir mil errores a mano? ¿Y diez mil? Automatizar las cosas lo antes posible es generalmente una buena idea. Además, ¡es divertido! :)

> **Coincidencia difusa:** El proceso de buscar automáticamente cadenas de texto que son muy similares a la cadena de destino. En general, una cadena se considera "más cercana" a otra, menos caracteres necesitaría cambiar si estuviera transformando una cadena en otra. Entonces, "apple" y "snapple" están a dos cambios uno del otro (agregue "s" y "n") mientras que "in" y "on" y un cambio de distancia (rplace "i" con "o"). No siempre podrá confiar en la coincidencia aproximada al 100%, pero por lo general terminará ahorrándole al menos un poco de tiempo.

Fuzzywuzzy devuelve una proporción dadas dos cadenas. Cuanto más cercana sea la relación a 100, menor será la distancia de edición entre las dos cadenas. Aquí, vamos a obtener las diez cadenas de nuestra lista de ciudades que tienen la distancia más cercana a "d.i khan".

In [ ]:
# get the top 10 closest matches to "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

Podemos ver que dos de los elementos en las ciudades están muy cerca de "d.i khan": "d. I khan" y "d.i khan". También podemos ver que "dg khan", que es una ciudad separada, tiene una proporción de 88. Como no queremos reemplazar "dg khan" por "di khan", reemplacemos todas las filas en nuestra columna Ciudad que tienen una relación de> 90 con "d. i khan".

Para hacer esto, voy a escribir una función. (Es una buena idea escribir una función de propósito general que pueda reutilizar si cree que podría tener que hacer una tarea específica más de una o dos veces. Esto evita que tenga que copiar y pegar código con demasiada frecuencia, lo que ahorra tiempo y puede ayudar prevenir errores.)

In [ ]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

Ahora que tenemos una función, ¡podemos ponerla a prueba!

In [ ]:
# use the function we just wrote to replace close matches to "d.i khan" with "d.i khan"
replace_matches_in_column(df=suicide_attacks, column='City', string_to_match="d.i khan")

Y ahora podemos verificar los valores únicos en nuestra columna Ciudad nuevamente y asegurarnos de que hemos ordenado d.i khan correctamente.

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

¡Excelente! Ahora solo tenemos "d.i khan" en nuestro marco de datos y no tuvimos que cambiar nada a mano.

In [ ]:
# Your turn! It looks like 'kuram agency' and 'kurram agency' should
# be the same city. Correct the dataframe so that they are.

¡Y eso es todo por hoy! Si tiene alguna pregunta, asegúrese de publicarla en los comentarios a continuación o [on the forums](https://www.kaggle.com/questions-and-answers). 

Recuerde que su bloc de notas es privado de forma predeterminada y, para compartirlo con otras personas o pedir ayuda con él, deberá hacerlo público. Primero, deberá guardar una versión de su cuaderno que muestre su trabajo actual presionando el botón "Confirmar y ejecutar". (Su trabajo se guarda automáticamente, pero el control de versiones de su trabajo le permite volver atrás y ver cómo era en el momento en que lo guardó. También le permite compartir un buen cuaderno compilado en lugar de solo el código sin procesar). la computadora portátil ha terminado de ejecutarse, puede ir a la pestaña Configuración en el panel de la izquierda (es posible que deba expandirla presionando el botón [<] junto al botón "Confirmar y ejecutar") y configurar el menú desplegable "Visibilidad" en "Público".

# ¡Más práctica!
___

¿Alguna otra columna en este marco de datos tiene entrada de datos inconsistente? Si puede encontrar alguno, intente ordenarlo.

También puede intentar leer en el `PakistanSuicideAttacks Ver 6 (10-October-2017).csv` archivo de este conjunto de datos y ordenar las columnas inconsistentes en ese archivo de datos